Dati un file di poligoni (land.geojson) e un file di punti (farms.geojson):
* verifica se land.geojson contiene geometrie multipolygon, le esplode in geometri di tipo polygon e salva il risultato in poly_exploded.geojson
* verifica la proiezione di poly_exploded.geojson e farms.geojson, la imposta come 3857 e salva i risultati come poly_exploded_projected.geojson e farms_projected.geojson nella cartella temporanea

In [86]:
import os
import geopandas as gpd
from pyproj import Transformer

# Carica il file di poligoni e verifica se ci sono geometrie multipolygon
polys = gpd.read_file(os.path.join('data','land.geojson'))
if polys.geom_type.any() == 'MultiPolygon':
    # Esplode le geometrie multipolygon in poligoni singoli
    polys = polys.explode()

# Verifica e imposta la proiezione dei dati
if polys.crs != 'epsg:3857':
    transformer = Transformer.from_crs(polys.crs, 'epsg:3857')
    polys = polys.dissolve().to_crs('epsg:3857')

farms = gpd.read_file(os.path.join('data','farms.geojson'))
if farms.crs != 'epsg:3857':
    transformer = Transformer.from_crs(farms.crs, 'epsg:3857')
    farms = farms.to_crs('epsg:3857')

# Salva i risultati in file separati
polys.to_file(os.path.join('tmp','poly_exploded_projected.geojson'), driver='GeoJSON')
farms.to_file(os.path.join('tmp','farms_projected.geojson'), driver='GeoJSON')

Prosegue:

* crea buffer di 100 metri di poly_exploded_projected.geojson usando una risoluzione di 3 punti e salva il risultato con buffer.geojson;
* crea buffer di 5 km intorno a farms_projected.geojson;
* utilizzando una espressione trigonometrica, verifica quali i vertici del poligono buffer.geojson hanno un angolo esterno pari o superiore a 180 gradi e li salva in un file vertici_convessi.geojson


In [87]:
import geopandas as gpd
from shapely.geometry import Polygon, LineString, Point
import math
import os

# Carica il file dei poligoni esplodi e verifica la proiezione
polys = gpd.read_file(os.path.join('tmp','poly_exploded_projected.geojson'))
if polys.crs != 'epsg:3857':
    polys = polys.to_crs('epsg:3857')

# Crea un buffer di 100 metri attorno ai poligoni
buffer = polys.buffer(500, join_style=2)

# Salva il buffer in un file separato
buffer.to_file(os.path.join('tmp','buffer.geojson'), driver='GeoJSON')

# Carica il file delle farm e verifica la proiezione
farms = gpd.read_file(os.path.join('tmp','farms_projected.geojson'))
if farms.crs != 'epsg:3857':
    farms = farms.to_crs('epsg:3857')

# Crea un buffer di 5 km attorno alle farm e salva in un file separato
farms_buffer = farms.buffer(5000)
farms_buffer.to_file(os.path.join('tmp','farms_buffer.geojson'), driver='GeoJSON')
farms_buffer_df = gpd.GeoDataFrame(geometry=farms_buffer, crs='epsg:3857')

# Crea una lista vuota per i vertici convessi
convex_vertices = []

# Loop sui poligoni nel buffer
for polygon in buffer.explode().geometry:
    # Itera sui vertici del poligono
    for i in range(len(polygon.exterior.coords)):
        p1 = polygon.exterior.coords[i-1]
        p2 = polygon.exterior.coords[i]
        p3 = polygon.exterior.coords[(i+1) % len(polygon.exterior.coords)]
        if (p2[0]-p1[0])*(p3[1]-p2[1]) - (p3[0]-p2[0])*(p2[1]-p1[1]) < 0:
            convex_vertices.append(p2)

# Crea un GeoDataFrame contenente i vertici convessi
convex_points = gpd.GeoDataFrame(geometry=gpd.points_from_xy([point[0] for point in convex_vertices], [point[1] for point in convex_vertices]), crs='EPSG:3857')
# Salva i vertici convessi in un file separato come punti
convex_points.to_file(os.path.join('tmp','vertici_convessi.geojson'), driver='GeoJSON')

# Seleziona solo i punti contenuti nel buffer intorno alle farms
convex_points_aoi = convex_points[convex_points.geometry.within(farms_buffer_df.unary_union)] 
print('vertici convessi:', len(convex_points))
print('vertici convessi contenuti nel buffer degli allevamenti:', len(convex_points_aoi))
# Salva il file dei vertici convessi nell'area di interesse
convex_points_aoi.to_file(os.path.join('tmp','vertici_convessi_aoi.geojson'), driver='GeoJSON')

C:\Users\a.dilorenzo\AppData\Local\Temp\ipykernel_17648\821732671.py:31: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  for polygon in buffer.explode().geometry:


vertici convessi: 703
vertici convessi contenuti nel buffer degli allevamenti: 528


aggiungere l'attributo "FARM_CODE" a ogni punto del file vertici_convessi_aoi.geojson, unire i punti di farms_projected.geojson con quelli del file vertici_convessi_aoi.geojson e salvare il risultato in un file graph_nodes.geojson

In [88]:
import geopandas as gpd

# Carica il file dei vertici convessi nell'area di interesse
convex_points_aoi = gpd.read_file(os.path.join('tmp','vertici_convessi_aoi.geojson'))

# Aggiungi l'attributo FARM_CODE a ogni punto del file
convex_points_aoi['FARM_CODE'] = 'vertex'

# Carica il file delle fattorie
farms = gpd.read_file(os.path.join('tmp','farms_projected.geojson'))

# Unisci i punti dei due file
graph_nodes = farms.append(convex_points_aoi)

# Salva il file dei nodi del grafo
graph_nodes.to_file(os.path.join('tmp','graph_nodes.geojson'), driver='GeoJSON')

print('graph nodes:', len(graph_nodes))

graph nodes: 543


C:\Users\a.dilorenzo\AppData\Local\Temp\ipykernel_17648\2923304900.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  graph_nodes = farms.append(convex_points_aoi)


Dati i file graph_nodes.geojson e poly_proiected.geojson:
utilizzare la libreria networkx per creare una rete di visibilità che unica tra loro tutti i punti node_network_projected.geojson considerando i poligoni di poly_projected.gejson come ostacoli che non possono essere attraversati dalle linee di connessione
salvare il risultato come graph_network.geojson

In [89]:
import geopandas as gpd
import networkx as nx
from shapely.geometry import Point, LineString, MultiPolygon

# Carica i file dei nodi del grafo e dei poligoni
nodes = gpd.read_file(os.path.join('tmp','graph_nodes.geojson'))
polygons = gpd.read_file(os.path.join('tmp','poly_exploded_projected.geojson'))

# Crea il grafo
G = nx.Graph()

# Aggiungi i nodi al grafo
for i, row in nodes.iterrows():
    G.add_node(i, geometry=row.geometry)

# Calcola le connessioni tra i nodi visibili
for u, udata in G.nodes(data=True):
    for v, vdata in G.nodes(data=True):
        if u >= v:
            continue
        # Crea la linea tra i due nodi
        line = LineString([udata['geometry'].coords[0], vdata['geometry'].coords[0]])
        visible = True
        # Verifica se la linea attraversa un poligono
        for _, poly in polygons.iterrows():
            if line.intersects(poly.geometry):
                visible = False
                break
        # Aggiungi la connessione se i due nodi sono visibili
        if visible:
            G.add_edge(u, v)

# Crea un nuovo GeoDataFrame con le linee di connessione
edges = []
for u, v, data in G.edges(data=True):
    edge = LineString([nodes.loc[u].geometry.coords[0], nodes.loc[v].geometry.coords[0]])
    edges.append({'id': f"{u}-{v}", 'geometry': edge})
edges_gdf = gpd.GeoDataFrame(edges, crs='epsg:3857')

# Salva il file delle linee di connessione come geojson
edges_gdf.to_file(os.path.join('tmp','graph_network.geojson'), driver='GeoJSON')